# K-Nearest Neighbors Classification

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/105477/pages/python-and-jupyter-notebooks?module_item_id=2613813) on Canvas (under Modules> Coding Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. Deep learning libraries such as keras and pytorch are also allowed.  You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  


<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it. We reserve the right to deduct up to 2 marks for unreadable or exessively inefficient code.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board (Piazza -> Assignment_1); we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. Please check the <a href="https://canvas.lms.unimelb.edu.au/courses/105477/modules">CIS Academic Honesty training</a> for more information. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this assignment, you will implement the K-nearest neighbor (KNN) classification algorithm and apply it to a real-world machine learning data set. In particular we will classify zoo animals into seven animal categories. 

Firstly, you will read in the dataset into a train and a test set (Q1). Secondly, you will implement different distance functions (Q2). Thirdly, you will implement a KNN classifier (Q3). You will apply and evaluate your classifier on the data set exploring different parameters (Q4, Q5). Finally, you will critically discuss your results (Q6).

### Question 1: Loading the data

**Instructions:** For this assignment we will develop a K-Nearest Neighbors (KNN) classifier to classify animals in the zoo into pre-defined categories of animals, namely
```
1: mammal
2: bird
3: reptile
4: fish
5: amphibian
6: insect
7: invertebrate
```

We use a data set of zoo animals from the UCI Machine learning repository.

The original data can be found here: https://archive.ics.uci.edu/ml/datasets/Zoo. 

The dataset consists of 101 instances. Each instance corresponds to an animal which has a unique identifier (the name of the animal; first field) and is characterized with 16 features:

```
   1. hair		    Boolean
   2. feathers		Boolean
   3. eggs		    Boolean
   4. milk		    Boolean
   5. airborne		Boolean
   6. aquatic		Boolean
   7. predator		Boolean
   8. toothed		Boolean
   9. backbone		Boolean
  10. breathes		Boolean
  11. venomous		Boolean
  12. fins		    Boolean
  13. legs		    Numeric (set of values: {0,2,4,5,6,8})
  14. tail		    Boolean
  15. domestic		Boolean
  16. catsize		Boolean
```

You need to first obtain this dataset, which is on Canvas (assignment 1). The files we will be using are called *zoo.data* and *zoo.labels*. Make sure the files are saved in the same folder as this notebook.

Both files are in comma-separated value format.

*zoo.features* contains 101 instances, one line per instance. The first field is the unique instance identifier (name of animals). The following fields contain the 16 features, as described above.

*zoo.labels* contains the gold labels (i.e., one of the seven classes above), for one instance per line. Again, the first field is the instance identifier, and the second field the instance label.

**Task**: Read the two files  
1. create a **training_feature** set (list of features for the first 90 instances in the zoo.* files) and a **training_label** set (list of labels for the corresponding). 
2. create a **test_feature** set (list of features of the remaining instances in the zoo.* files) and a **test_label** set (list of labels for the corresponding). 

**Check**: Use the assertion statements in *"For your testing"* to validate your code.

In [1]:
data = open("zoo.features.csv", 'r').readlines()
gold = open("zoo.labels.csv", 'r').readlines()

train_features = []
train_labels   = []
test_features = []
test_labels   = []

###########################
## YOUR CODE BEGINS HERE
###########################
for i in range(len(data)):
    data[i] = data[i].strip()
    data[i] = data[i].split(',')
    
    if i < 90:
        train_features.append(data[i][1:])
    else:
        test_features.append(data[i][1:])

for i in range(len(train_features)):
    train_features[i] = [int(x) for x in train_features[i]]
    
for j in range(len(test_features)):
    test_features[j] = [int(y) for y in test_features[j]]  
    
    
for k in range(len(gold)):
    gold[k] = gold[k].strip()
    gold[k] = gold[k].split(',')
    
    if k < 90:
        train_labels.append(int(gold[k][1]))
    else:
        test_labels.append(int(gold[k][1]))

###########################
## YOUR CODE BEGINS HERE
###########################

<b>For your testing</b>

In [2]:
assert len(train_features) == len(train_labels)
assert len(train_features[0])==len(train_features[-1])
assert train_features[2][12]==0 and train_labels[2]==4

### Question 2: Distance Functions

<b>Instructions</b>: Implement the four distance functions specified below. Use <b>only</b> the library imported below, i.e., <b>do not</b> use implementations from any other Python library. 

1. Eucledian distance
2. Cosine distance
3. Hamming distance 
4. Jaccard distance 

Each distance function takes as input
- Two feature vectors

and returns as output
- The distance between the two feature vectors (float)

**Note** for the purpose of this assignment we consider the numeric feature (legs) to be discretized already with each individual value belonging constituting a separate class.

In [3]:
import math

#########################
# Your answer BEGINS HERE
#########################
   

def eucledian_distance(fv1, fv2):
    # insert code here
    distance = math.sqrt(sum([(a-b)**2 for a, b in zip(fv1, fv2)]))
    d = round(float(distance), 2)
    return d

def cosine_distance(fv1, fv2):
    # insert code here
    
    sum_aa = 0 #a square
    sum_bb = 0 #b square
    sum_ab = 0 #a * b
    
    for i in range(len(fv1)):
        a = fv1[i]
        b = fv2[i]
        sum_aa += a*a
        sum_bb += b*b
        sum_ab += a*b
        
    return 1 - (sum_ab / math.sqrt(sum_aa*sum_bb))

def hamming_distance(fv1, fv2):
    # insert code here
    dist = 0
    for i in range(0,len(fv1)):
        if fv1[i] != fv2[i]:
            dist += 1
    return dist

#print (hamming_distance([1,1,1,1], [0,1,0,0]))
    
    
def jaccard_distance(fv1, fv2):   
    # insert code here
    intersection = 0
    count_fv1 = 0
    count_fv2 = 0
    
    for i in range(0, len(fv1)):
        if fv1[i] != 0 and fv2[i] != 0:
            if fv1[i] == fv2[i]:
                intersection += 1
       
    for j in range(0, len(fv1)):
        if fv1[j] != 0:
            count_fv1 += 1
            
    for k in range(0, len(fv2)):
        if fv2[k] != 0:
            count_fv2 += 1
            
    union = (count_fv1 + count_fv2) - intersection
    #print('intersection')
    #print(intersection)
    return 1 - (float(intersection) / union)

#jaccard_distance([1,1,1,1], [0,1,0,0])


In [4]:
assert round(eucledian_distance([1,0],[0.5,1]),2)==1.12 
assert jaccard_distance([1,1,1,1], [0,1,0,0])==0.75

<b>For your testing: </b>

### Question 3: KNN Classifier

<b>Instructions</b>: Here, you implement your KNN classifier. It takes as input 
- training data features
- training data labels
- test data features
- parameter K
- distance function(s) based on which nearest neighbors will be identified

It returns as output 
- the predicted labels for the test data

**Voting stragegy** Your classifier will use majority voting (i.e., no distance weighting)

**You should implement the classifier from scratch yourself**, i.e., <b> you must not</b> use an existing implementation in any Python library.

**Ties**. Ties may occur when computing the K nearest neighbors, or when predicting the class based on the neighborhood. You may deal with ties whichever way you want (as long as you still use the requested distance functions).

#### <b>For your testing:</b>

In [5]:
def KNN(train_features, train_labels, test_features, k, dist_fun):
    
    predictions = []
    
    ###########################
    ## YOUR CODE BEGINS HERE
    ###########################
       
    for test in test_features:
        #calculate the distances between each test data and training data
        distances = []
        for i in range(0, len(train_features)):
            d = dist_fun(train_features[i],test)
            distances.append(d)
        
        #############TEST##############
        #print('distances')
        #print(distances)       
        #############TEST##############
        
        s = sorted(range(len(distances)), key = lambda d : distances[d])   # sort the distances
        
        #############TEST##############
        #print('return indexes of sorted distances')
        #print(s)
        #print(len(s))
        #############TEST##############
        
        neighbours = []      #find the neighbours based on k and save them/it into neighbours list
        for i in range(k):
            neighbours.append(s[i])
         
        vote_values = []             #now count the vote
        for neighbour in neighbours:
            vote_values.append(neighbour)

        predict_index = max(set(vote_values), key = vote_values.count)
        
        predict = train_labels[predict_index]
        predictions.append(predict)
    #############TEST##############
    #print('--------')
    #print(predictions)
    #############TEST##############

    ###########################
    ## YOUR CODE BEGINS HERE
    ########################### 
    
    return predictions  


In [6]:
assert KNN([[1,1],[5,5],[1,2]], [1,0,1], [[1,1]], 1, eucledian_distance) == [1]

### Question 4: Evaluation
**Instructions:** Write a function that computes the "accuracy" of your classifier. Given a set of predicted lables and a set of gold labels, it returns the fraction of correct labels over all predicted labels. 

**Example**: `The gold truth labels for four test instances are [1, 1, 1, 1]. A system predicted the labels [0, 1, 0, 0] for the same 4 instances. The accuracy of the system is 1/4 = 0.25
`

Your function will take as input 
- gold labels
- predicted labels

It returns as output
- the accuracy value (float).


In [7]:
def accuracy(predict, gold):
    
    ###########################
    ## YOUR CODE BEGINS HERE
    ###########################
    correct = 0
    for i in range(len(gold)):
        if gold[i] == predict[i]:
            correct += 1
    accuracy = round(correct / float(len(gold)), 2)


    ###########################
    ## YOUR CODE BEGINS HERE
    ###########################

    return accuracy

<b>For your testing:</b>

In [8]:
assert accuracy([1, 1, 1, 1], [0, 1, 0, 1])==0.5

### Question 5: Applying your KNN classifiers to the Zoo Dataset

**Using the functions you have implemented above, please**

<b> 1. </b>
For each of the distance functions you implemented in Question 2, construct four KNN classifiers with K=1, K=5, K=25, K=55. You will create a total of 16 (4 distance functions x 4 K values) classifiers.

<b> 2. </b>
Compute the test accuracy for each model

In [9]:
########################
# Your code STARTS HERE
########################

# 1. Predict test labels with each KNN classifier
k = [1, 5, 25, 55]
#dist_fun_list = [eucledian_distance, cosine_distance, jaccard_distance,hamming_distance]

result_ed = []
result_cd = []
result_jd = []
result_hd = []

for i in range(0, len(k)):
    #for j in range(0, len(dist_fun_list)):
    result_ed.append(KNN(train_features, train_labels, test_features, k[i], eucledian_distance))
    result_cd.append(KNN(train_features, train_labels, test_features, k[i], cosine_distance))
    result_jd.append(KNN(train_features, train_labels, test_features, k[i], jaccard_distance))
    result_hd.append(KNN(train_features, train_labels, test_features, k[i], hamming_distance))
    
    
#print(KNN(train_features, train_labels, test_features, 5, cosine_distance))
##########TEST#############    
#print('ed')
#print(result_ed)
#print('cd')
#print(result_cd)
#print('jd')
#print(result_jd)
#print('hd')
#print(result_hd)
##########TEST#############    

# 2. Compute the accuracy scores 

for i in range(0 ,len(result_ed)):
    accuracy_ed = accuracy(result_ed[i], test_labels)
    
for i in range(0 ,len(result_cd)):
    accuracy_cd = accuracy(result_cd[i], test_labels)
    
for i in range(0 ,len(result_ed)):
    accuracy_jd = accuracy(result_jd[i], test_labels)
    
for i in range(0 ,len(result_ed)):
    accuracy_hd = accuracy(result_hd[i], test_labels)

accuracy_knn_euc_1 = accuracy(result_ed[0], test_labels)
accuracy_knn_euc_5 = accuracy(result_ed[1], test_labels)
accuracy_knn_euc_25 = accuracy(result_ed[2], test_labels)
accuracy_knn_euc_55 = accuracy(result_ed[3], test_labels)


accuracy_knn_cos_1 = accuracy(result_cd[0], test_labels)
accuracy_knn_cos_5 = accuracy(result_cd[1], test_labels)
accuracy_knn_cos_25 = accuracy(result_cd[2], test_labels)
accuracy_knn_cos_55 = accuracy(result_cd[3], test_labels)

accuracy_knn_jac_1 = accuracy(result_jd[0], test_labels)
accuracy_knn_jac_5 = accuracy(result_jd[1], test_labels)
accuracy_knn_jac_25 = accuracy(result_jd[2], test_labels)
accuracy_knn_jac_55 = accuracy(result_jd[3], test_labels)

accuracy_knn_ham_1 = accuracy(result_hd[0], test_labels)
accuracy_knn_ham_5 = accuracy(result_hd[1], test_labels)
accuracy_knn_ham_25 = accuracy(result_hd[2], test_labels)
accuracy_knn_ham_55 = accuracy(result_hd[3], test_labels)



########################
# Your code ENDS HERE
########################

print("Euclidean")
print(accuracy_knn_euc_1)
print(accuracy_knn_euc_5)
print(accuracy_knn_euc_25)
print(accuracy_knn_euc_55)
print("Cosine")
print(accuracy_knn_cos_1)
print(accuracy_knn_cos_5)
print(accuracy_knn_cos_25)
print(accuracy_knn_cos_55)
print("Jaccard")
print(accuracy_knn_jac_1)
print(accuracy_knn_jac_5)
print(accuracy_knn_jac_25)
print(accuracy_knn_jac_55)
print("Hamming")
print(accuracy_knn_ham_1)
print(accuracy_knn_ham_5)
print(accuracy_knn_ham_25)
print(accuracy_knn_ham_55)


Euclidean
0.82
0.82
0.64
0.45
Cosine
0.82
0.73
0.55
0.45
Jaccard
0.82
0.82
0.64
0.36
Hamming
0.82
0.91
0.64
0.36
